## loading buildings data

In [1]:
import pandas as pd

banepa_buildings = pd.read_csv("./banepa_buildings.csv")
banepa_buildings

/var/folders/zz/qg9mqbp56631kxmjjyl11f1h0000gn/T/ipykernel_4121/883645039.py:3: DtypeWarning: Columns (0,3,8,9,11,12,17,21,22,23,24,25,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,55,57,60,61,62,65,67,68,69,70,71,72,73,74,75,76,78,79,80,81,82,84,85,86,87,88,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  banepa_buildings = pd.read_csv("./banepa_buildings.csv")


,addr:housenumber,building,name,old_name,geometry,roof:material,roof:shape,amenity,name:en,name:ne,...,material,sloar_panel,garden:type,solar:panel,product,ways,type,centroid,latitude,longitude
0,6,house,krishan prasad niwas,banepa,POINT (85.5207728 27.6293612),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (85.5207728 27.6293612),27.629361,85.520773
1,NaN,temple,चन्देश्वरी मन्दिर,NaN,POINT (85.5178667 27.6338529),NaN,NaN,place_of_worship,chandeshwori temple,चन्देश्वरी मन्दिर,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (85.5178667 27.6338529),27.633853,85.517867
2,NaN,yes,Nagarkot Geodetic Observatory,NaN,POINT (85.5210844 27.6928248),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (85.5210844 27.6928248),27.692825,85.521084
3,NaN,yes,NaN,NaN,POINT (85.515002 27.6278862),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (85.515002 27.6278862),27.627886,85.515002
4,NaN,yes,NaN,NaN,POINT (85.5234071 27.6366944),tin,flat,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (85.5234071 27.6366944),27.636694,85.523407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22521,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4903851 27.6398999, 85.4904...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (85.49044215344654 27.639867467729527),27.639867,85.490442
22522,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4859034 27.6356064, 85.4858...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (85.48596051398037 27.635566587340723),27.635567,85.485961
22523,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4923976 27.6357972, 85.4924...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (85.49246051461394 27.63577735590748),27.635777,85.492461
22524,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.5131996 27.6454771, 85.5132...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"[1083556229, 344763945]",multipolygon,POINT (85.51315060244725 27.645406309834463),27.645406,85.513151


## loading landmarks

In [2]:
import h3
from sklearn.cluster import DBSCAN
import numpy as np

class LandmarkPriority:

    def __init__(self, hex_resolution=7):
        self.hex_resolution = hex_resolution

    def get_hexagons(self, lat, lon):
        h3_index = h3.geo_to_h3(lat, lon, self.hex_resolution)
        return h3.k_ring(h3_index, 1)

    def cluster_landmarks(self, landmarks):
        coords = np.array([(landmark['lat'], landmark['lon']) for landmark in landmarks])
        # coords = landmarks_df[['lat', 'lon']].values  # Extract only lat and lon columns as a numpy array
        clustering = DBSCAN(eps=0.005, min_samples=5).fit(coords)
        labels = clustering.labels_
        return labels

    def rank_clusters(self, labels):
        unique, counts = np.unique(labels, return_counts=True)
        cluster_counts = dict(zip(unique, counts))
        # Sort clusters by size
        sorted_clusters = sorted(cluster_counts.items(), key=lambda x: x[1], reverse=True)
        return sorted_clusters

    def select_priority_landmark(self, landmarks, labels):
        sorted_clusters = self.rank_clusters(labels)
        top_cluster_label = sorted_clusters[0][0]
        top_cluster_landmarks = [landmarks[i] for i in range(len(labels)) if labels[i] == top_cluster_label]

        # Prioritization logic
        priority_order = ["temple", "tourist_spot", "bus_stop", "government_building", "market", "school"]
        for priority in priority_order:
            for landmark in top_cluster_landmarks:
                if landmark['type'] == priority:
                    return landmark

        # In case no landmark matches the prioritization logic
        return top_cluster_landmarks[0]

    def get_priority_landmark_for_hex(self, lat, lon, landmarks):
        hexagons = self.get_hexagons(lat, lon)
        landmarks_in_hex = [landmark for landmark in landmarks if h3.geo_to_h3(landmark['lat'], landmark['lon'], self.hex_resolution) in hexagons]
        if not landmarks_in_hex:
            return None
        labels = self.cluster_landmarks(landmarks_in_hex)
        priority_landmark = self.select_priority_landmark(landmarks_in_hex, labels)
        return priority_landmark

In [3]:
import pandas as pd

landmarks = pd.read_csv('landmarks_clean.csv')
landmarks = landmarks[["tags_name", "lat", "lon", "tags_amenity"]]
landmarks.columns = ["name", "lat", "lon", "type"]

landmarks_dict = landmarks.to_dict('records')

In [4]:


banepa_buildings.latitude = banepa_buildings.latitude.astype(float)
banepa_buildings.longitude = banepa_buildings.longitude.astype(float)
landmark_tag = []

# visualize with tqdm

for lat, lon in zip(banepa_buildings.latitude, banepa_buildings.longitude):
    landmark_priority = LandmarkPriority()
    priority_landmark = landmark_priority.get_priority_landmark_for_hex(lat, lon, landmarks_dict)
    landmark_tag.append(priority_landmark)
    print(priority_landmark)

{'name': 'नेपाल बनेपा पोलिटेक्निक संस्थान', 'lat': 27.6367561, 'lon': 85.5246034, 'type': 'school'}
{'name': 'नेपाल बनेपा पोलिटेक्निक संस्थान', 'lat': 27.6367561, 'lon': 85.5246034, 'type': 'school'}
{'name': 'Shree Ratna Chundeshwor Secondary School', 'lat': 27.6731607, 'lon': 85.5164054, 'type': 'school'}
{'name': 'नेपाल बनेपा पोलिटेक्निक संस्थान', 'lat': 27.6367561, 'lon': 85.5246034, 'type': 'school'}
{'name': 'नेपाल बनेपा पोलिटेक्निक संस्थान', 'lat': 27.6367561, 'lon': 85.5246034, 'type': 'school'}
{'name': 'नेपाल बनेपा पोलिटेक्निक संस्थान', 'lat': 27.6367561, 'lon': 85.5246034, 'type': 'school'}
{'name': 'नेपाल बनेपा पोलिटेक्निक संस्थान', 'lat': 27.6367561, 'lon': 85.5246034, 'type': 'school'}
{'name': 'नेपाल बनेपा पोलिटेक्निक संस्थान', 'lat': 27.6367561, 'lon': 85.5246034, 'type': 'school'}
{'name': 'नेपाल बनेपा पोलिटेक्निक संस्थान', 'lat': 27.6367561, 'lon': 85.5246034, 'type': 'school'}
{'name': 'नेपाल बनेपा पोलिटेक्निक संस्थान', 'lat': 27.6367561, 'lon': 85.5246034, 'type': '

In [5]:
import pandas as pd
import numpy as np

landmark_tag = [i if i else {"name": np.nan, "lat": np.nan, "lon": np.nan, "type": np.nan} for i in landmark_tag]
landmark_tag_df = pd.DataFrame(landmark_tag)
landmark_tag_df.columns = ["landmark_" + i for i in landmark_tag_df.columns]

# Handle NaN values in the lambda function
landmark_tag_df["landmark_name"] = landmark_tag_df["landmark_name"].apply(
    lambda x: "_".join(i.strip() for i in x.split(" ")) if pd.notna(x) else x
)

banepa_buildings = pd.concat([banepa_buildings, landmark_tag_df], axis=1)


In [6]:
banepa_buildings

,addr:housenumber,building,name,old_name,geometry,roof:material,roof:shape,amenity,name:en,name:ne,...,product,ways,type,centroid,latitude,longitude,landmark_name,landmark_lat,landmark_lon,landmark_type
0,6,house,krishan prasad niwas,banepa,POINT (85.5207728 27.6293612),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,POINT (85.5207728 27.6293612),27.629361,85.520773,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school
1,NaN,temple,चन्देश्वरी मन्दिर,NaN,POINT (85.5178667 27.6338529),NaN,NaN,place_of_worship,chandeshwori temple,चन्देश्वरी मन्दिर,...,NaN,NaN,NaN,POINT (85.5178667 27.6338529),27.633853,85.517867,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school
2,NaN,yes,Nagarkot Geodetic Observatory,NaN,POINT (85.5210844 27.6928248),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,POINT (85.5210844 27.6928248),27.692825,85.521084,Shree_Ratna_Chundeshwor_Secondary_School,27.673161,85.516405,school
3,NaN,yes,NaN,NaN,POINT (85.515002 27.6278862),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,POINT (85.515002 27.6278862),27.627886,85.515002,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school
4,NaN,yes,NaN,NaN,POINT (85.5234071 27.6366944),tin,flat,NaN,NaN,NaN,...,NaN,NaN,NaN,POINT (85.5234071 27.6366944),27.636694,85.523407,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22521,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4903851 27.6398999, 85.4904...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,POINT (85.49044215344654 27.639867467729527),27.639867,85.490442,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school
22522,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4859034 27.6356064, 85.4858...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,POINT (85.48596051398037 27.635566587340723),27.635567,85.485961,जनज्योति_माध्यमिक_विद्यालय,27.632557,85.499560,school
22523,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4923976 27.6357972, 85.4924...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,POINT (85.49246051461394 27.63577735590748),27.635777,85.492461,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school
22524,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.5131996 27.6454771, 85.5132...",NaN,NaN,NaN,NaN,NaN,...,NaN,"[1083556229, 344763945]",multipolygon,POINT (85.51315060244725 27.645406309834463),27.645406,85.513151,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school


# Hashing

In [29]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import math
from math import radians, cos, sin, asin, sqrt, atan2
import osmnx as ox
                
G = ox.graph_from_place('Banepa, Nepal', network_type='drive')

def find_nearest_node(G, point):
    return min(G.nodes, key=lambda x: math.dist((G.nodes[x]['y'], G.nodes[x]['x']), point))

class RouteOptimizer:

    def __init__(self, G, landmark_location, destination_location):
        self.G = G
        self.landmark_location = find_nearest_node(G, landmark_location)
        self.destination_location = find_nearest_node(G, destination_location)
        
    def get_shortest_path(self):
        # impute missing edge speed and add travel times
        self.G = ox.add_edge_speeds(self.G)
        self.G = ox.add_edge_travel_times(self.G)

        # calculate shortest path minimizing travel time
        orig = self.landmark_location
        dest = self.destination_location
        self.path = nx.shortest_path(self.G, orig, dest, 'travel_time')
        return self.path
    
    
    def calculate_initial_compass_bearing(self, pointA, pointB):
        lat1 = math.radians(pointA[0])
        lat2 = math.radians(pointB[0])
        diffLong = math.radians(pointB[1] - pointA[1])
        x = math.sin(diffLong) * math.cos(lat2)
        y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(diffLong))
        initial_bearing = math.atan2(x, y)
        initial_bearing = math.degrees(initial_bearing)
        compass_bearing = (initial_bearing + 360) % 360
        return compass_bearing


def haversine_distance(coord1, coord2, round_to=0):
    R = 6371000  # radius of Earth in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = radians(lat1), radians(lat2) 
    dphi       = radians(lat2 - lat1)
    dlambda    = radians(lon2 - lon1)
    
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlambda/2)**2
    distance = 2 * R * atan2(sqrt(a), sqrt(1 - a))
    
    if round_to == 0:
        return int(distance)
    
    return round(float(distance), round_to)


# Hash Generation Function
def generate_hash(landmark_name, path, optimizer):
    directions = []
    prev_dir = None
    sum_length = 0

    for i in range(len(path) - 1):
        # Retrieve the coordinates of the nodes from the graph
        pointA = (optimizer.G.nodes[path[i]]['y'], optimizer.G.nodes[path[i]]['x'])
        pointB = (optimizer.G.nodes[path[i + 1]]['y'], optimizer.G.nodes[path[i + 1]]['x'])
        
        bearing = optimizer.calculate_initial_compass_bearing(pointA, pointB)
   
        if i == len(path) - 2:
            print("Last point")
            length = haversine_distance(pointA, pointB, round_to=2)
        else:
            length = haversine_distance(pointA, pointB)
        
        if (bearing >= 0 and bearing < 45) or (bearing >= 315 and bearing < 360):
            dir = "N"
        elif bearing >= 45 and bearing < 135:
            dir = "E"
        elif bearing >= 135 and bearing < 225:
            dir = "S"
        else:
            dir = "W"

        if prev_dir == dir:
            sum_length += length
        else:
            if prev_dir is not None:
                directions.append(f"{prev_dir}_{sum_length}")
            sum_length = length
            prev_dir = dir

    # Add the last direction and length
    if prev_dir is not None:
        directions.append(f"{prev_dir}_{sum_length}")
        
    # make the last direction round to 2 decimal places
    try:
        directions[-1] = f"{directions[-1].split('_')[0]}_{float(directions[-1].split('_')[1]):.2f}"
    except:
        pass

    hash_string = f"{landmark_name}|{'|'.join(directions)}"
    return hash_string
    

In [30]:
# Example usage:
destination_location = (27.632028, 85.53)
landmark_location = [27.632278, 85.501588]


optimizer = RouteOptimizer(G, landmark_location, destination_location)
path = optimizer.get_shortest_path()
# print(path)
hash_string = generate_hash('Landmark1', path, optimizer)
print(f"For building at (30.702, 90.305), hash_string is: {hash_string}")

For building at (30.702, 90.305), hash_string is: Landmark1|E_2505.56|N_100.38|E_54.78|N_64.2|E_288.44


In [31]:
# delete rows in banepa_buildings where landmark_name is null
banepa_buildings = banepa_buildings[banepa_buildings.landmark_name.notnull()]
banepa_buildings

,addr:housenumber,building,name,old_name,geometry,roof:material,roof:shape,amenity,name:en,name:ne,...,ways,type,centroid,latitude,longitude,landmark_name,landmark_lat,landmark_lon,landmark_type,hashes
0,6,house,krishan prasad niwas,banepa,POINT (85.5207728 27.6293612),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.5207728 27.6293612),27.629361,85.520773,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_401|W_126|N_...
1,NaN,temple,चन्देश्वरी मन्दिर,NaN,POINT (85.5178667 27.6338529),NaN,NaN,place_of_worship,chandeshwori temple,चन्देश्वरी मन्दिर,...,NaN,NaN,POINT (85.5178667 27.6338529),27.633853,85.517867,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...
2,NaN,yes,Nagarkot Geodetic Observatory,NaN,POINT (85.5210844 27.6928248),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.5210844 27.6928248),27.692825,85.521084,Shree_Ratna_Chundeshwor_Secondary_School,27.673161,85.516405,school,Shree_Ratna_Chundeshwor_Secondary_School|W_275...
3,NaN,yes,NaN,NaN,POINT (85.515002 27.6278862),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.515002 27.6278862),27.627886,85.515002,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_401|W_126|N_...
4,NaN,yes,NaN,NaN,POINT (85.5234071 27.6366944),tin,flat,NaN,NaN,NaN,...,NaN,NaN,POINT (85.5234071 27.6366944),27.636694,85.523407,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22521,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4903851 27.6398999, 85.4904...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.49044215344654 27.639867467729527),27.639867,85.490442,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...
22522,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4859034 27.6356064, 85.4858...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.48596051398037 27.635566587340723),27.635567,85.485961,जनज्योति_माध्यमिक_विद्यालय,27.632557,85.499560,school,जनज्योति_माध्यमिक_विद्यालय|W_828|N_235|W_591.01
22523,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4923976 27.6357972, 85.4924...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.49246051461394 27.63577735590748),27.635777,85.492461,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...
22524,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.5131996 27.6454771, 85.5132...",NaN,NaN,NaN,NaN,NaN,...,"[1083556229, 344763945]",multipolygon,POINT (85.51315060244725 27.645406309834463),27.645406,85.513151,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...


In [32]:
# iterate over the rows of the banepe_buildings dataframe and generate the hash
hashes = []
for index, row in banepa_buildings.iterrows():
    landmark_location = [row.landmark_lat, row.landmark_lon]
    destination_location = (row.latitude, row.longitude)
    optimizer = RouteOptimizer(G, landmark_location, destination_location)
    path = optimizer.get_shortest_path()
    hash_string = generate_hash(row.landmark_name, path, optimizer)
    hashes.append(hash_string)
    print(f"Index: {index} :: Building at ({row.latitude}, {row.longitude}) has hash: {hash_string}")

Index: 0 :: Building at (27.6293612, 85.5207728) has hash: नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_402.13|W_127.79|N_23.63|W_147.63000000000002|S_213.29999999999998|E_11.96|S_140.67
Index: 1 :: Building at (27.6338529, 85.5178667) has hash: नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302.62|N_111.07|W_63.07|S_175.85|W_265.52000000000004|S_7.26|W_41.73
Index: 2 :: Building at (27.6928248, 85.5210844) has hash: Shree_Ratna_Chundeshwor_Secondary_School|W_276.28|N_1695.42
Index: 3 :: Building at (27.6278862, 85.515002) has hash: नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_402.13|W_127.79|N_23.63|W_223.54000000000002|S_217.67|W_87.62|S_46.03|W_128.62|S_10.27|W_111.9|S_317.53
Index: 4 :: Building at (27.6366944, 85.5234071) has hash: नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302.62|N_111.07|W_53.89|N_118.28|E_22.7|N_137.06
Index: 5 :: Building at (27.6222856, 85.5306402) has hash: नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_374.06|E_75.35|S_383.64000000000004|E_647.38|S_162.08|E_7.37|S_268.53
Index: 6 :: Building at (27.6224062, 85.53

KeyboardInterrupt: 

In [ ]:
banepa_buildings.to_csv("banepa_buildings_with_hashes_float.csv", index=False)

In [20]:
len(banepa_buildings["hashes"])

22474

In [18]:
json_data = banepa_buildings[["name", "latitude", "longitude", "landmark_name", "landmark_lat", "landmark_lon", "hashes"]]
# write into json file in multiple lines
json_data.to_json("banepa_buildings_with_hashes.json", orient="records")

In [25]:
len(banepa_buildings.latitude.unique())

22459

In [28]:
sum(banepa_buildings["hashes"] == "नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_53|N_118|E_22.70")

18

In [22]:
len(banepa_buildings["hashes"].unique())

937

In [51]:
len(banepa_buildings["hashes"].unique())

Index(['addr:housenumber', 'building', 'name', 'old_name', 'geometry',
       'roof:material', 'roof:shape', 'amenity', 'name:en', 'name:ne',
       'religion', 'office', 'source', 'green_roof', 'roof:solar_panel',
       'roof:tin_roof', 'roof:water_tank', 'man_made', 'construction', 'nodes',
       'layer', 'generator:source', 'location', 'power', 'roof:type',
       'tourism', 'addr:street', 'shop', 'CONS_TYPE', 'building:levels',
       'building:use', 'building:structure', 'building:overhang',
       'building:soft_storey', 'seismic_resistance', 'water_tank', 'HH_LINK',
       'street', 'ROOF_TYPE', 'Cantilever', 'Engin_Noneng', 'ExpansionJoint',
       'House_type', 'Physical_condition', 'Pounding_Poss', 'ShortColumn',
       'Visible_GroundSettlemt', 'rooftopwatertank', 'soft:weakStorey',
       'CATEGORY', 'addr:hamlet', 'operator:type', 'safe_place', 'addr:city',
       'addr:postcode', 'opening_hours', 'phone', 'operator',
       'building:material', 'leisure', 'note', 'sport

In [52]:
banepa_buildings

,addr:housenumber,building,name,old_name,geometry,roof:material,roof:shape,amenity,name:en,name:ne,...,ways,type,centroid,latitude,longitude,landmark_name,landmark_lat,landmark_lon,landmark_type,hashes
0,6,house,krishan prasad niwas,banepa,POINT (85.5207728 27.6293612),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.5207728 27.6293612),27.629361,85.520773,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_402|W_127|N_...
1,NaN,temple,चन्देश्वरी मन्दिर,NaN,POINT (85.5178667 27.6338529),NaN,NaN,place_of_worship,chandeshwori temple,चन्देश्वरी मन्दिर,...,NaN,NaN,POINT (85.5178667 27.6338529),27.633853,85.517867,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...
2,NaN,yes,Nagarkot Geodetic Observatory,NaN,POINT (85.5210844 27.6928248),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.5210844 27.6928248),27.692825,85.521084,Shree_Ratna_Chundeshwor_Secondary_School,27.673161,85.516405,school,Shree_Ratna_Chundeshwor_Secondary_School|W_276...
3,NaN,yes,NaN,NaN,POINT (85.515002 27.6278862),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.515002 27.6278862),27.627886,85.515002,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_402|W_127|N_...
4,NaN,yes,NaN,NaN,POINT (85.5234071 27.6366944),tin,flat,NaN,NaN,NaN,...,NaN,NaN,POINT (85.5234071 27.6366944),27.636694,85.523407,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22521,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4903851 27.6398999, 85.4904...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.49044215344654 27.639867467729527),27.639867,85.490442,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...
22522,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4859034 27.6356064, 85.4858...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.48596051398037 27.635566587340723),27.635567,85.485961,जनज्योति_माध्यमिक_विद्यालय,27.632557,85.499560,school,जनज्योति_माध्यमिक_विद्यालय|W_831|N_235|W_591
22523,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.4923976 27.6357972, 85.4924...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,POINT (85.49246051461394 27.63577735590748),27.635777,85.492461,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...
22524,NaN,yes,NaN,NaN,"MULTIPOLYGON (((85.5131996 27.6454771, 85.5132...",NaN,NaN,NaN,NaN,NaN,...,"[1083556229, 344763945]",multipolygon,POINT (85.51315060244725 27.645406309834463),27.645406,85.513151,नेपाल_बनेपा_पोलिटेक्निक_संस्थान,27.636756,85.524603,school,नेपाल_बनेपा_पोलिटेक्निक_संस्थान|S_302|N_111|W_...
